## 🔮 बोल  भिडू ! 

In [89]:
import cohere
import streamlit as st

co = cohere.Client(st.secrets.COHERE_API_KEY)
# pc = Pinecone(api_key=st.secrets.PINECONE_API_KEY)


### 1. Loading document

In [71]:
with open("C:/Users/mayur dabade/Desktop/Projects/marathi RAG/data/maharaj.txt", encoding='utf-8') as f:
    text = f.read()

print(f"The text has roughly {len(text.split())} words.")

The text has roughly 1030 words.


### 2. Splitting doc into chunks

In [72]:
# For chunking let's use langchain to help us split the text
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [73]:
# Create basic configurations to chunk the text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# Split the text into chunks with some overlap
chunks_ = text_splitter.create_documents([text])
chunks = [c.page_content for c in chunks_]
print(f"The text has been broken down in {len(chunks)} chunks.")

The text has been broken down in 15 chunks.


### 3. Embed every text chunk


In [74]:
# Because the texts being embedded are the chunks we are searching over, we set the input type as search_doc
model="embed-multilingual-v3.0"
response = co.embed(
    texts= chunks,
    model=model,
    input_type="search_document",
    embedding_types=['float']
)
embeddings = response.embeddings.float
print(f"We just computed {len(embeddings)} embeddings.")

We just computed 15 embeddings.


### Storing id, chunks and embeddings to pinecone

In [75]:
# import os
# from pinecone import Pinecone, ServerlessSpec

# # Initialize Pinecone with your API key
# api_key = os.getenv("PINECONE_API_KEY", "6727bdcd-c680-4056-9398-64bedaaee775")
# pc = Pinecone(api_key=api_key)

# # Create a serverless index
# # Ensure "dimension" matches the dimensions of the vectors you upsert
# pc.create_index(name="products", dimension=len(embeddings[0]), 
#                 spec=ServerlessSpec(cloud='aws', region='us-east-1'))

# # Target the index
# index = pc.Index("products")

# # Prepare the vectors and metadata for upsert
# vectors_to_upsert = []
# for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
#     vector = {
#         "id": f"vector_{i}",
#         "values": embedding,
#         "metadata": {"description": chunk}
#     }
#     vectors_to_upsert.append(vector)

# # Upsert vectors into the Pinecone index
# index.upsert(vectors=vectors_to_upsert)

# print("Vectors successfully upserted into Pinecone!")


### 4. Storing ids and embeddings to pinecone

In [76]:
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone with your API key
pc = Pinecone(api_key=st.secrets.PINECONE_API_KEY)

# Create a serverless index
# Ensure "dimension" matches the dimensions of the vectors you upsert
pc.create_index(name="product", dimension=len(embeddings[0]), 
                spec=ServerlessSpec(cloud='aws', region='us-east-1'))

# Target the index
index = pc.Index("product")

# Prepare the vectors for upsert (only id and embedding)
vectors_to_upsert = []
for i, embedding in enumerate(embeddings):
    vector = {
        "id": f"{i}",
        "values": embedding
    }
    vectors_to_upsert.append(vector)

# Upsert vectors into the Pinecone index
index.upsert(vectors=vectors_to_upsert)

print("Vectors successfully upserted into Pinecone!")


Vectors successfully upserted into Pinecone!


### 5. Given a user query, retrieve the relevant chunks from the vector database


### Define the user question

In [77]:
query = "सगळ्यात जास्त शेअर असणाऱ्या भारतातील कंपनी?"
# query="सीसीआयने या मर्जरला काही अटींसह मंजुरी का दिली? या अटींचा उद्देश काय असू शकतो?"
# query = "मुघल व आदिलशाही फौजांचा यशस्वीपणे सामना"


### 6. Embed the user question


In [78]:
# Because the text being embedded is the search query, we set the input type as search_query
response = co.embed(
    texts=[query],
    model=model,
    input_type="search_query",
    embedding_types=['float']
)
query_embedding = response.embeddings.float[0]
# print("query_embedding: ", query_embedding)

### 7. Retrieve the most relevant chunks from the vector database

We use cosine similarity to find the most similar chunks

In [79]:
index = pc.Index("product")

query_results1 = index.query(
    # namespace="example-namespace1",
    vector=query_embedding,
    top_k=3,
    include_values=True
)
# query_results1

# Extracting id and score
result = [{'id': match['id'], 'score': match['score']} for match in query_results1['matches']]

print(result)

[{'id': '4', 'score': 0.63326323}, {'id': '11', 'score': 0.611728072}, {'id': '1', 'score': 0.609839082}]


In [80]:
ids = [int(item['id']) for item in result]
print(ids)

for i in ids:
    print("\n",chunks[i])

[4, 11, 1]

 या या कंपनीत सर्वाधिक वर्चस्व हे रिलायन्स असणार आहे या कंपनीमध्ये डिजनी कड 3684% ंची भागीदारी असेल तर उरलेला 75% वाटा हा स्टार इंडियाचे सीईओ उदय शंकर आणि जेम्स मुरदोज यांच्या बोधी ट्री या जॉईंट व्हेंचर कडे असणार आहे मात्र या एकत्रीकरणासाठी कोर्टाद्वारे मान्यता प्राप्त संस्थेची परवानगी असणं गरजेचं आहे या परवानगी नंतरच या दोन कंपन्या एकत्र येऊ शकतात आता या डील साठी भारतातील कॉम्पिटिशन कमिशन ऑफ इंडिया म्हणजे सीसीआय ची परवानगी असणं आवश्यक होतं सीसीआय ही वैधानिक संस्था भारतीय बाजारपेठेतील स्पर्धा टिकवून ठेवणं

 सगळ्यात जास्त शेअर असणाऱ्या भारतातील डिजनी आणि आक्रमकपणे या क्षेत्रात आगेकू करणाऱ्या रिलायन्स च्या एकत्र येण्यामुळं भारतातील मीडिया क्षेत्रात एका जायंट मीडिया कंपनीचा जन्म होणं ही मोठी चिंतेची बाब असल्याचं बोललं जातंय आता येत्या काळात जेव्हा क्रिकेट सामन्यांचे प्रक्षेपण केलं जाईल तेव्हा क्रिकेटच्या दरम्यान ज्या जाहिराती दाखवल्या जातात त्यांचे दर हे इतर कोणताही स्पर्धक नसल्यामुळे खूप जास्त असतील त्यामुळे या जाहिरात कंपन्यांची बार्गेनिंग पावर ही कमी होईल याचा परिणाम जाहिरा

### Matching Chunks for the user query

In [81]:
context = "\n\n".join(chunks[i] for i in ids)
# print(context)

### 8. designing final answer

In [82]:


template = f"""Use the following pieces of context to answer the user question. This context retrieved from a knowledge base and you should use only the facts from the context to answer.
Your answer must be based on the context. If the context not contain the answer, just say that 'I don't know', don't try to make up an answer, use the context.
Don't address the context directly, but use it to answer the user question like it's your own knowledge.
Use three sentences maximum. answer should be in Marathi.

Context:
{context}

Question: {query}
"""


In [85]:
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass(st.secrets.GOOGLE_API_KEY)

llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke(template)
print(result.content)

सगळ्यात जास्त शेअर असणारी भारतीय कंपनी ही रिलायन्स आहे.


In [84]:
import numpy as np

# Create a NumPy array
arr = np.array(chunks)

# Save the array to a .npy file
np.save('my_array.npy', arr)


In [88]:
import numpy as np

# Load the array from the .npy file
arr = np.load('my_array.npy')

# Verify the loaded array
print(len(arr))


15
